Taking User Input

In [ ]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')



corpus=[]

for i in range(0, 100):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [2]:
user_input = input("Type a situation to know its sentiment")

In [12]:
#data transformation - feature detection
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
tfidfFile='C:\\Users\\user\\Desktop\\SentimentAnalysisUsingNLP\\tfidf_Sentiment_Model.pk1'
tfidf = pickle.load(open(tfidfFile, "rb"))

In [ ]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

In [ ]:
#prediction using classifier
import joblib
classifier = joblib.load('LR_Classifier_Sentiment_Model')

In [ ]:
y_pred = classifier.predict(X_fresh)
print(y_pred)